In [7]:
import pandas as pd
import gdown

### Loading data

In [8]:
# Downloading data from Google Drive.
file_id = "1a4YBBG-QsmFOvDDtnpqHvXJRz6iy-OaG"
url = f"https://drive.google.com/uc?id={file_id}"
file_extension = "downloaded_file.json"
gdown.download(url, file_extension, quiet=False)

df = pd.read_json(file_extension, lines=True)

# Dropping unnecessary columns:
# '6' has been split on 2 columns due to an error during the download,
# while '8' is a duplicate of '9' with minor differences. However, '9' is
# believed to be the correct version.
df.drop(columns=[6,8], inplace=True)

# Renaming columns.
df.columns = [
    'Ad ID', 'Job title', 'Employer (entity)', 'Employer location', 'Office address', 'Workplace', 'Department',
    'Posted on', 'Expiration date', 'Detailed recruitment results', 'Salary (monthly)', 'Vacancies', 'Full-time ratio',
    'Recruitment results', 'Description of duties', 'Required level of education', 'Prerequisites',
    'Optional requirements', 'Application instructions', 'Number of views'
]

#print(df['Recruitment results'].nunique())

# The data contains no active posts at the time of the download:
# the column "Recruitment results" contains only one unique value - "Koniec naboru"
# (recruitment finished).
# Thus, we can drop the column.
df.drop(columns=["Recruitment results"], inplace=True)

Downloading...
From (original): https://drive.google.com/uc?id=1a4YBBG-QsmFOvDDtnpqHvXJRz6iy-OaG
From (redirected): https://drive.google.com/uc?id=1a4YBBG-QsmFOvDDtnpqHvXJRz6iy-OaG&confirm=t&uuid=3e6fd919-353b-44fd-a6ec-ca56a51f7c18
To: C:\Users\dede\Desktop\Magisterka\Kod\2024-mgr-sluzba-cywilna\Python\Code\downloaded_file.json
100%|██████████| 335M/335M [00:14<00:00, 23.7MB/s] 


### Reducing data size

In [9]:
# We're removing unnecessary text from rows, such as "NR" (an abbreviation for number)
# in every cell of the "Ad ID" column.

for index, row in df.iterrows():
    if row['Ad ID'][0] == 'N' and row['Ad ID'][2] == ' ':
        df.at[index, 'Ad ID'] = row['Ad ID'][3:]

    # Removing "Adres urzędu: " from each row.
    if row['Office address'][12] == ':' and row['Office address'][13] == ' ':
        df.at[index, 'Office address'] = row['Office address'][14:]
    elif row['Office address'][12] == ':':
        df.at[index, 'Office address'] = row['Office address'][13:]

    # Removing "Miejsce wykonywania pracy: " from each row.
    if row['Workplace'][25] == ':' and row['Workplace'][26] == ' ':
        df.at[index, 'Workplace'] = row['Workplace'][27:]
    elif row['Workplace'][25] == ':':
        df.at[index, 'Workplace'] = row['Workplace'][26:]

    # Removing "Wykształcenie: " from each row.
    if row['Required level of education'][13] == ':' and row['Required level of education'][14] == ' ':
        df.at[index, 'Required level of education'] = row['Required level of education'][15:]
    elif row['Required level of education'][13] == ':':
        df.at[index, 'Required level of education'] = row['Required level of education'][14:]

    # Removing "Wyniki naboru: " from each row.
    if row['Detailed recruitment results'][14] == ' ' and row['Detailed recruitment results'][13] == ':':
        df.at[index, 'Detailed recruitment results'] = row['Detailed recruitment results'][15:]
    elif row['Detailed recruitment results'][13] == ':':
        df.at[index, 'Detailed recruitment results'] = row['Detailed recruitment results'][14:]

    # Removing "Liczba odwiedzin: " from each row.
    # One additional condition since the first character in most entries is a space.
    if row['Required level of education'][0] == ' ':
        if row['Number of views'][18] == ':' and row['Number of views'][19] == ' ':
            df.at[index, 'Number of views'] = row['Number of views'][20:]
        elif row['Number of views'][18] == ':' and row['Number of views'][1].lower() == 'l':
            df.at[index, 'Number of views'] = row['Number of views'][19:]
    else:
        if row['Number of views'][17] == ':' and row['Number of views'][18] == ' ':
            df.at[index, 'Number of views'] = row['Number of views'][19:]
        elif row['Number of views'][16] == ':' and row['Number of views'][0].lower() == 'l':
            df.at[index, 'Number of views'] = row['Number of views'][17:]

### Data split

In [10]:
# Splitting the data and exploring the frequency of different recruitment results.

# Announcements that ended with the employment of a candidate.
df_success = df[df['Detailed recruitment results'].str.contains(
    r'wyborem kandydatki/kandydata|informacja o zatrudnieniu kandydatki|zatrudnieniem'
)][['Detailed recruitment results']]
success_count = len(df_success)

# Announcements that ended without employing a candidate.
# Jak skategoryzować ogłoszenia, które w rezultacie mają wpisane
# "decyzja o rezygnacji kandydata z obejmowanego stanowiska"?
# Na razie są wrzucone jako fail, aczkolwiek jest to dyskusyjne.
df_failure = df[df['Detailed recruitment results'].str.contains(r'bez wyboru kandydatki/kandydata|nie wybrano|nie zatrudniono|rak aplikacji|rak zgłoszeń|rak ofert|rak kandydatów|rak zapisu|kandydaci zrezygnowali|ikt się nie zgłosił|rak kandydatur|rak wniosków|bez zatrudnienia|kandydat zrezygnował|nie spełniały wymagań|nie wyłoniono|rezygnacji|rezygnacja|kandydat zrezygnował|zrezygnował z|nie zgłosił się|andydaci nie przybyli|zrezygnowali|braku kandydatur|kandydatka zrezygnowała|ezygnacja kandydatki|żadna oferta|rak złożonych ofert|ezygnacja kandydata|nadesłanych ofert|braku kandydatów|braku kandydatur')][['Detailed recruitment results']]
failure_count = len(df_failure)


# Announcements that have been cancelled as a results of an error; they are omitted in our analysis.
df_error1 = df[df['Detailed recruitment results'].str.contains(r'łędu|łąd|łędem|łędne|orekta ogłoszenia|łędnie|łędny|łędy w ogłoszeniu|rak informacji|ie uwzględniono|łędna|omyłkowo|omyłka|nie uwwzględniono|przypadkowo|dwukrotnie|podwójnie|razy wprowadzono te same|zdublowane|problem techniczny|miana treści|wymaga korekty')][['Detailed recruitment results']]
error1_count = len(df_error1)

df_error2 = df.loc[df['Detailed recruitment results'] == 'anulowano nabór|']
error2_count = len(df_error2)

error_count = error1_count + error2_count
print(f"Ads that led to employment: {success_count}\n"
      f"Ads that did not lead to employment: {failure_count}\n"
      f"Ads cancelled as a result of an error: {error1_count}")


# A relatively small amount of ads was left, and they're not a subject of our analysis (e.g. a decision to cancel a vacancy or the return of an employee to work).
# From now on, we're going to operate only on ads present in failure_df and success_df.
df_failure = df[df.index.isin(list(df_failure.index))]
df_success = df[df.index.isin(list(df_success.index))]
dfs_of_interest = [df_failure, df_success]


# Comparing frequency of "no salary given" in failure and success.
df_failure_no_salary = df_failure[df_failure['Salary (monthly)'].str.contains('nie podano')][['Salary (monthly)']]
df_success_no_salary = df_success[df_success['Salary (monthly)'].str.contains('nie podano')][['Salary (monthly)']]
failure_no_salary_count = len(df_failure_no_salary)
success_no_salary_count = len(df_success_no_salary)

failure_no_salary_percentage = failure_no_salary_count/failure_count
success_no_salary_percentage = success_no_salary_count/success_count

print(f"Percentage of no salary in df_success: {success_no_salary_percentage}\n"
      f"Percentage of no salary in df_failure: {failure_no_salary_percentage}")

Ads that led to employment: 58659
Ads that did not lead to employment: 43199
Ads cancelled as a result of an error: 464
Percentage of no salary in df_success: 0.7137353176835609
Percentage of no salary in df_failure: 0.7684668626588579


### Making data readable for computers

In [11]:
# 1. The salary column
#First, we're checking whether all salaries are given as gross (brutto)
# or if there are some given as net (netto).
df_netto = df[df['Salary (monthly)'].str.contains('netto')][['Salary (monthly)']]
count_netto = len(df_netto)
#print(count_netto)
# There are only 100 salaries given in netto.

# 2. The required level of education column.
# Converting required level of educations to hierarchical categorical variables:
# 1 - high school education
# 2 - specific profile of high school education (e.g. "średnie weterynaryjne").
# 3 - higher education
# 4 - specific profile of higher education (e.g. "wyższe ekonomiczne").
# There are no ads that would require less than high school education.

for df_of_interest in dfs_of_interest:
    for index, row in df_of_interest.iterrows():
        if row["Required level of education"][0:7] == "średnie":
            if len(row["Required level of education"]) > 10:
                df_of_interest.at[index, "Required level of education"] = 1
            else:
                df_of_interest.at[index, "Required level of education"] = 2
        elif row["Required level of education"][0:6] == "wyższe":
            if len(row["Required level of education"]) > 9:
                df_of_interest.at[index, "Required level of education"] = 3
            else:
                df_of_interest.at[index, "Required level of education"] = 4

#dfs_of_interest[0]["Required level of education"].unique()
#dfs_of_interest[1]["Required level of education"].unique()
# The columns in both dfs contain only values defined by us above.

In [14]:
# Saving the dataframe to place it on Google Drive.
df_output = pd.concat([df_success, df_failure], ignore_index=True, axis=0)
df_output.to_csv("initially_cleansed_data.csv", index=False)

In [16]:
df_output.head(10)

,Ad ID,Job title,Employer (entity),Employer location,Office address,Workplace,Department,Posted on,Expiration date,Detailed recruitment results,Salary (monthly),Vacancies,Full-time ratio,Description of duties,Required level of education,Prerequisites,Optional requirements,Application instructions,Number of views
0,130994,inspektor weterynaryjny,Powiatowy Inspektorat Weterynarii w Oleśnie,Olesno,ul. Kossaka 5 46-300 Olesno,Olesno,"w Zespole ds. bezpieczeństwa żywności, pasz i ...",14 grudnia 2023,12 grudnia 2023,nabór zakończony wyborem kandydatki/kandydata|...,nie podano wynagrodzenia,1,1/2,[Sprawuje nadzór nad działalnością TPDW w zakr...,3,[doświadczenie zawodowe: co najmniej 6 miesięc...,"[Przeszkolenie w zakiesie pobierania prób., Zn...","[Dokumenty należy złożyć do: 12.12.2023, Decyd...",183
1,130898,asystent,Powiatowy Inspektorat Weterynarii w Chrzanowie,Chrzanów,Mjr Grzybowskiego 7 32-500 Chrzanów,Chrzanów,w zespole ds. administracyjnych,13 grudnia 2023,11 grudnia 2023,nabór zakończony wyborem kandydatki/kandydata|...,"4400,00 zł brutto",1,1,"[Przyjmuje i ewidencjonuje korespondencję, prz...",2,"[Prawo jazdy kat. B, Umiejętność obsługi kompu...",[Wykształcenie: wyższe ekonomiczne lub adminis...,"[Dokumenty należy złożyć do: 11.12.2023, Decyd...",701
2,130837,inspektor weterynaryjny,Powiatowy Inspektorat Weterynarii w Świdnicy,Świdnica,ul. Wałbrzyska 25-27 58-100 Świdnica,Świdnica,Zespół ds. zdrowia i ochrony zwierząt,13 grudnia 2023,12 grudnia 2023,nabór zakończony wyborem kandydatki/kandydata|...,nie podano wynagrodzenia,1,1,[wykonuje czynności z zakresu zwalczania choró...,3,"[prawo wykonywania zawodu lekarza weterynarii,...",[czynne prawo jazdy kat. B],"[Dokumenty należy złożyć do: 12.12.2023, Decyd...",228
3,130818,radca,Ministerstwo Finansów w Warszawie,Warszawa,ul. Świętokrzyska 12 00-916 Warszawa,Warszawa,w Wydziale Prognoz Makroekonomicznych i Analiz...,15 grudnia 2023,8 grudnia 2023,nabór zakończony wyborem kandydatki/kandydata|...,nie podano wynagrodzenia,1,1,"[Inicjuje prace koncepcyjno-metodologiczne, ko...",3,[doświadczenie zawodowe: co najmniej 4 lata w ...,[Znajomość metodologii Europejskiego Systemu R...,"[Dokumenty należy złożyć do: 08.12.2023, Decyd...",934
4,130730,starszy referent,Izba Administracji Skarbowej w Rzeszowie,Krosno,ul. Geodetów 1 35-959 Rzeszów,Krosno US w Krośnie ul. Składowa 5,w Wieloosobowym Stanowisku Podatków Majątkowyc...,13 grudnia 2023,4 grudnia 2023,nabór zakończony wyborem kandydatki/kandydata|...,"nie mniej niż 4972,00 zł brutto",1,1,[Obsługuje i wspiera podatnika i płatnika w pr...,2,"[doświadczenie zawodowe: powyżej 0,5 roku w ad...","[CV i list motywacyjny, Kopie dokumentów potwi...","[Dokumenty należy złożyć do: 04.12.2023, Decyd...",778
5,130808,główny księgowy,Komenda Powiatowa Państwowej Straży Pożarnej w...,Kamień Pomorski,Wolińska 7d 72-400 Kamień Pomorski,Kamień Pomorski,Samodzielne stanowisko pracy ds. finansowych,13 grudnia 2023,8 grudnia 2023,nabór zakończony wyborem kandydatki/kandydata|...,nie podano wynagrodzenia,1,1,"[1.\tprowadzenie rachunkowości jednostki,, 2.\...",1,[doświadczenie zawodowe: 3 lata doświadczenia ...,[Wykształcenie: wyższe wyższe na kierunku rach...,"[Dokumenty należy złożyć do: 08.12.2023, Decyd...",214
6,130588,starszy referent,Izba Administracji Skarbowej w Rzeszowie,Rzeszów,ul. Geodetów 1 35-959 Rzeszów,Rzeszów Drugi US w Rzeszowie ul. Siemieńskiego 18,w Dziale Obsługi Bezpośredniej,13 grudnia 2023,29 listopada 2023,nabór zakończony wyborem kandydatki/kandydata|...,"nie mniej niż 4972,00 zł brutto",1,1,[Dokonuje czynności sprawdzających i weryfikac...,2,"[doświadczenie zawodowe: powyżej 0,5 roku w ad...","[CV i list motywacyjny, Kopie dokumentów potwi...","[Dokumenty należy złożyć do: 29.11.2023, Decyd...",1116
7,130596,starszy kontroler skarbowy,Izba Administracji Skarbowej w Białymstoku,Białystok,ul. Jana Klemensa Branickiego 9 15-085 Białystok,"Białystok Białystok, Świętojańska 13",w Drugim Dziale Obsługi Bezpośredniej w Pierws...,13 g